In [1]:
import requests

In [8]:
url = "https://cluster.elasticsearch.dataesr.ovh/bso-clinical-trials/_search"
headers = { "Authorization": open("../../.env_es_bsso", "r").read().strip() }
AGG_SIZE = 15


### Sommaire

* [2. Essais cliniques](#publications)
    * [2.1. Général](#publi_general)
        * [2.1.1. Quelle est la dynamique dd'ouverture des essais ?](#dynamique_ouverture)
        * [2.1.2. Quelles sont les trajectoires des essais ?](#trajectoires)

# 2. Essais cliniques <a class="anchor" id="publications"></a>

## 2.1. Général <a class="anchor" id="publi_general"></a>

### 2.1.1. Quelle est la dynamique d’ouverture des essais ? <a class="anchor" id="dynamique_ouverture"></a>

In [19]:
# manque l'aggreg par type de lead sponsor
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "study_type.keyword": "Interventional"
        }
      }]
    }
  },
  "aggs": {
    "by_year": {
      "terms": {
        "field": "study_start_year"
      },
      "aggs": {
        "by_has_results": {
          "terms": {
            "field": "has_results" # has_results_or_publications
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_year']['buckets']

[{'key': 2019.0,
  'doc_count': 1566,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 1548},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 18}]}},
 {'key': 2018.0,
  'doc_count': 1512,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 1437},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 75}]}},
 {'key': 2015.0,
  'doc_count': 1470,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 1151},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 319}]}},
 {'key': 2020.0,
  'doc_count': 1443,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 1439},
    {'key': 1, 'key_as_string': 'true', 

In [23]:
### 2.1.2. Quelles sont les trajectoires? <a class="anchor" id="trajectoire"></a>

In [20]:
# manque l'aggreg has_publications_result et has_publication_oa
json = {
  "size": 0,
  "query": {
    "bool": {
      "filter": [{
        "term": {
          "study_type.keyword": "Interventional"
        }
      }]
    }
  },
  "aggs": {
    "by_status": {
      "terms": {
        "field": "status.keyword"
      },
      "aggs": {
        "by_has_results": {
          "terms": {
            "field": "has_results"
          }
        }
      }
    }
  }
}

requests.post(url, json=json, headers=headers).json()['aggregations']['by_status']['buckets']

[{'key': 'Completed',
  'doc_count': 10876,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 7435},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 3441}]}},
 {'key': 'Recruiting',
  'doc_count': 3948,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 3946},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 2}]}},
 {'key': 'Ongoing',
  'doc_count': 2464,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 1718},
    {'key': 1, 'key_as_string': 'true', 'doc_count': 746}]}},
 {'key': 'Unknown status',
  'doc_count': 2047,
  'by_has_results': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 0, 'key_as_string': 'false', 'doc_count': 2042},
    {'key': 1,